<h2 style="text-align: center;">Bezier Surface Patch</h2>
<h3 style="text-align: center;">Ross Beveridge, December 10, 2017</h3>
<p style="text-align: left;">This notebook sets up the linear algebra to create a bicubic Bezier Surface Patch.  Keep in mind that a Bezier Surface patch is created by essentially combining Bezier curves in one parameter, the first dimension, with a second Bezier curve in a separate parameter, the second dimension.  Always be careful to distinguish the dimensionality of an object versus the space into which it is embedded.  A Bezier Surface Patche is 2D - that is why two scalars are needed to specify location on the surface.  However, because of our formulation, it lives in 3D space (is embedded in); and the tip off is that for every pair of parameters there are three distinct coordinates. </p>

In [1]:
%display latex
latex.matrix_delimiters(left='|', right='|')
latex.vector_delimiters(left='[', right=']')
var('u,v')
TV = Matrix(SR, 4,1, ((u**3,u**2,u,1)))
SV = Matrix(SR, 4,1, ((v**3,v**2,v,1)))
MB = Matrix(ZZ, 4,4, ((-1,3,-3,1),(3,-6,3,0),(-3,3,0,0),(1,0,0,0)))

<h3 style="text-align: center;">The Symbolic Version</h3>
<p>Symbolic math pagackes are a blessing and curse; you can quickly build and display very large constructs including polynomials.  </p>
<p>First, we see here how the X, Y, and Z coordinates of the control points segragate nicely; only the X coordinates influece the bicubic term for X and similarlly for Y and Z.  Also, see that expressed as a row vector times a matrix times a matrix times a matrix times a column vector the entire equation may be written in a moderatly compact form.  </p>

In [2]:
var('x11,x12,x13,x14,x21,x22,x23,x24,x31,x32,x33,x34,x41,x42,x43,x44')
var('y11,y12,y13,y14,y21,y22,y23,y24,y31,y32,y33,y34,y41,y42,y43,y44')
var('z11,z12,z13,z14,z21,z22,z23,z24,z31,z32,z33,z34,z41,z42,z43,z44')
GX = Matrix(SR, 4,4, (x11,x12,x13,x14,x21,x22,x23,x24,x31,x32,x33,x34,x41,x42,x43,x44))
GY = Matrix(SR, 4,4, (y11,y12,y13,y14,y21,y22,y23,y24,y31,y32,y33,y34,y41,y42,y43,y44))
GZ = Matrix(SR, 4,4, (z11,z12,z13,z14,z21,z22,z23,z24,z31,z32,z33,z34,z41,z42,z43,z44))
pretty_print(LatexExpr('Q_{x}(u,v) = '),SV.transpose(), MB.transpose(), GX, MB, TV)
pretty_print(LatexExpr('Q_{y}(u,v) = '),SV.transpose(), MB.transpose(), GY, MB, TV)
pretty_print(LatexExpr('Q_{z}(u,v) = '),SV.transpose(), MB.transpose(), GZ, MB, TV)

Q_{x}(u,v) =  [v^3 v^2   v   1] [-1  3 -3  1]
[ 3 -6  3  0]
[-3  3  0  0]
[ 1  0  0  0] [x11 x12 x13 x14]
[x21 x22 x23 x24]
[x31 x32 x33 x34]
[x41 x42 x43 x44] [-1  3 -3  1]
[ 3 -6  3  0]
[-3  3  0  0]
[ 1  0  0  0] [u^3]
[u^2]
[  u]
[  1]

Q_{y}(u,v) =  [v^3 v^2   v   1] [-1  3 -3  1]
[ 3 -6  3  0]
[-3  3  0  0]
[ 1  0  0  0] [y11 y12 y13 y14]
[y21 y22 y23 y24]
[y31 y32 y33 y34]
[y41 y42 y43 y44] [-1  3 -3  1]
[ 3 -6  3  0]
[-3  3  0  0]
[ 1  0  0  0] [u^3]
[u^2]
[  u]
[  1]

Q_{z}(u,v) =  [v^3 v^2   v   1] [-1  3 -3  1]
[ 3 -6  3  0]
[-3  3  0  0]
[ 1  0  0  0] [z11 z12 z13 z14]
[z21 z22 z23 z24]
[z31 z32 z33 z34]
[z41 z42 z43 z44] [-1  3 -3  1]
[ 3 -6  3  0]
[-3  3  0  0]
[ 1  0  0  0] [u^3]
[u^2]
[  u]
[  1]

<p>The next three equations are correct by nature of how SageMath is constructing them.  Quickly can them to gain a general impression for what is taking place.  Then, recognized that while symbolic math packages allow us to see equations in the full glory (gory detail) there is too much going on to develop and intuition from the algebra. </p>

In [3]:
QX = (SV.transpose() * MB.transpose() * GX * MB * TV)[0][0]
QY = (SV.transpose() * MB.transpose() * GY * MB * TV)[0][0]
QZ = (SV.transpose() * MB.transpose() * GZ * MB * TV)[0][0]
QX = QX.expand()
QY = QY.expand()
QZ = QZ.expand()
pretty_print(LatexExpr('Q_{x}(u,v) = '), QX)
pretty_print(LatexExpr('Q_{y}(u,v) = '), QY)
pretty_print(LatexExpr('Q_{z}(u,v) = '), QZ)

Q_{x}(u,v) =  u^3*v^3*x11 - 3*u^3*v^3*x12 + 3*u^3*v^3*x13 - u^3*v^3*x14 - 3*u^3*v^3*x21 + 9*u^3*v^3*x22 - 9*u^3*v^3*x23 + 3*u^3*v^3*x24 + 3*u^3*v^3*x31 - 9*u^3*v^3*x32 + 9*u^3*v^3*x33 - 3*u^3*v^3*x34 - u^3*v^3*x41 + 3*u^3*v^3*x42 - 3*u^3*v^3*x43 + u^3*v^3*x44 - 3*u^3*v^2*x11 - 3*u^2*v^3*x11 + 9*u^3*v^2*x12 + 6*u^2*v^3*x12 - 9*u^3*v^2*x13 - 3*u^2*v^3*x13 + 3*u^3*v^2*x14 + 6*u^3*v^2*x21 + 9*u^2*v^3*x21 - 18*u^3*v^2*x22 - 18*u^2*v^3*x22 + 18*u^3*v^2*x23 + 9*u^2*v^3*x23 - 6*u^3*v^2*x24 - 3*u^3*v^2*x31 - 9*u^2*v^3*x31 + 9*u^3*v^2*x32 + 18*u^2*v^3*x32 - 9*u^3*v^2*x33 - 9*u^2*v^3*x33 + 3*u^3*v^2*x34 + 3*u^2*v^3*x41 - 6*u^2*v^3*x42 + 3*u^2*v^3*x43 + 3*u^3*v*x11 + 9*u^2*v^2*x11 + 3*u*v^3*x11 - 9*u^3*v*x12 - 18*u^2*v^2*x12 - 3*u*v^3*x12 + 9*u^3*v*x13 + 9*u^2*v^2*x13 - 3*u^3*v*x14 - 3*u^3*v*x21 - 18*u^2*v^2*x21 - 9*u*v^3*x21 + 9*u^3*v*x22 + 36*u^2*v^2*x22 + 9*u*v^3*x22 - 9*u^3*v*x23 - 18*u^2*v^2*x23 + 3*u^3*v*x24 + 9*u^2*v^2*x31 + 9*u*v^3*x31 - 18*u^2*v^2*x32 - 9*u*v^3*x32 + 9*u^2*v^2*x33 - 3*u*v^3*x41 + 3*u*v^3*x42 - u^3*x11 - 9*u^2*v*x11 - 9*u*v^2*x11 - v^3*x11 + 3*u^3*x12 + 18*u^2*v*x12 + 9*u*v^2*x12 - 3*u^3*x13 - 9*u^2*v*x13 + u^3*x14 + 9*u^2*v*x21 + 18*u*v^2*x21 + 3*v^3*x21 - 18*u^2*v*x22 - 18*u*v^2*x22 + 9*u^2*v*x23 - 9*u*v^2*x31 - 3*v^3*x31 + 9*u*v^2*x32 + v^3*x41 + 3*u^2*x11 + 9*u*v*x11 + 3*v^2*x11 - 6*u^2*x12 - 9*u*v*x12 + 3*u^2*x13 - 9*u*v*x21 - 6*v^2*x21 + 9*u*v*x22 + 3*v^2*x31 - 3*u*x11 - 3*v*x11 + 3*u*x12 + 3*v*x21 + x11

Q_{y}(u,v) =  u^3*v^3*y11 - 3*u^3*v^3*y12 + 3*u^3*v^3*y13 - u^3*v^3*y14 - 3*u^3*v^3*y21 + 9*u^3*v^3*y22 - 9*u^3*v^3*y23 + 3*u^3*v^3*y24 + 3*u^3*v^3*y31 - 9*u^3*v^3*y32 + 9*u^3*v^3*y33 - 3*u^3*v^3*y34 - u^3*v^3*y41 + 3*u^3*v^3*y42 - 3*u^3*v^3*y43 + u^3*v^3*y44 - 3*u^3*v^2*y11 - 3*u^2*v^3*y11 + 9*u^3*v^2*y12 + 6*u^2*v^3*y12 - 9*u^3*v^2*y13 - 3*u^2*v^3*y13 + 3*u^3*v^2*y14 + 6*u^3*v^2*y21 + 9*u^2*v^3*y21 - 18*u^3*v^2*y22 - 18*u^2*v^3*y22 + 18*u^3*v^2*y23 + 9*u^2*v^3*y23 - 6*u^3*v^2*y24 - 3*u^3*v^2*y31 - 9*u^2*v^3*y31 + 9*u^3*v^2*y32 + 18*u^2*v^3*y32 - 9*u^3*v^2*y33 - 9*u^2*v^3*y33 + 3*u^3*v^2*y34 + 3*u^2*v^3*y41 - 6*u^2*v^3*y42 + 3*u^2*v^3*y43 + 3*u^3*v*y11 + 9*u^2*v^2*y11 + 3*u*v^3*y11 - 9*u^3*v*y12 - 18*u^2*v^2*y12 - 3*u*v^3*y12 + 9*u^3*v*y13 + 9*u^2*v^2*y13 - 3*u^3*v*y14 - 3*u^3*v*y21 - 18*u^2*v^2*y21 - 9*u*v^3*y21 + 9*u^3*v*y22 + 36*u^2*v^2*y22 + 9*u*v^3*y22 - 9*u^3*v*y23 - 18*u^2*v^2*y23 + 3*u^3*v*y24 + 9*u^2*v^2*y31 + 9*u*v^3*y31 - 18*u^2*v^2*y32 - 9*u*v^3*y32 + 9*u^2*v^2*y33 - 3*u*v^3*y41 + 3*u*v^3*y42 - u^3*y11 - 9*u^2*v*y11 - 9*u*v^2*y11 - v^3*y11 + 3*u^3*y12 + 18*u^2*v*y12 + 9*u*v^2*y12 - 3*u^3*y13 - 9*u^2*v*y13 + u^3*y14 + 9*u^2*v*y21 + 18*u*v^2*y21 + 3*v^3*y21 - 18*u^2*v*y22 - 18*u*v^2*y22 + 9*u^2*v*y23 - 9*u*v^2*y31 - 3*v^3*y31 + 9*u*v^2*y32 + v^3*y41 + 3*u^2*y11 + 9*u*v*y11 + 3*v^2*y11 - 6*u^2*y12 - 9*u*v*y12 + 3*u^2*y13 - 9*u*v*y21 - 6*v^2*y21 + 9*u*v*y22 + 3*v^2*y31 - 3*u*y11 - 3*v*y11 + 3*u*y12 + 3*v*y21 + y11

Q_{z}(u,v) =  u^3*v^3*z11 - 3*u^3*v^3*z12 + 3*u^3*v^3*z13 - u^3*v^3*z14 - 3*u^3*v^3*z21 + 9*u^3*v^3*z22 - 9*u^3*v^3*z23 + 3*u^3*v^3*z24 + 3*u^3*v^3*z31 - 9*u^3*v^3*z32 + 9*u^3*v^3*z33 - 3*u^3*v^3*z34 - u^3*v^3*z41 + 3*u^3*v^3*z42 - 3*u^3*v^3*z43 + u^3*v^3*z44 - 3*u^3*v^2*z11 - 3*u^2*v^3*z11 + 9*u^3*v^2*z12 + 6*u^2*v^3*z12 - 9*u^3*v^2*z13 - 3*u^2*v^3*z13 + 3*u^3*v^2*z14 + 6*u^3*v^2*z21 + 9*u^2*v^3*z21 - 18*u^3*v^2*z22 - 18*u^2*v^3*z22 + 18*u^3*v^2*z23 + 9*u^2*v^3*z23 - 6*u^3*v^2*z24 - 3*u^3*v^2*z31 - 9*u^2*v^3*z31 + 9*u^3*v^2*z32 + 18*u^2*v^3*z32 - 9*u^3*v^2*z33 - 9*u^2*v^3*z33 + 3*u^3*v^2*z34 + 3*u^2*v^3*z41 - 6*u^2*v^3*z42 + 3*u^2*v^3*z43 + 3*u^3*v*z11 + 9*u^2*v^2*z11 + 3*u*v^3*z11 - 9*u^3*v*z12 - 18*u^2*v^2*z12 - 3*u*v^3*z12 + 9*u^3*v*z13 + 9*u^2*v^2*z13 - 3*u^3*v*z14 - 3*u^3*v*z21 - 18*u^2*v^2*z21 - 9*u*v^3*z21 + 9*u^3*v*z22 + 36*u^2*v^2*z22 + 9*u*v^3*z22 - 9*u^3*v*z23 - 18*u^2*v^2*z23 + 3*u^3*v*z24 + 9*u^2*v^2*z31 + 9*u*v^3*z31 - 18*u^2*v^2*z32 - 9*u*v^3*z32 + 9*u^2*v^2*z33 - 3*u*v^3*z41 + 3*u*v^3*z42 - u^3*z11 - 9*u^2*v*z11 - 9*u*v^2*z11 - v^3*z11 + 3*u^3*z12 + 18*u^2*v*z12 + 9*u*v^2*z12 - 3*u^3*z13 - 9*u^2*v*z13 + u^3*z14 + 9*u^2*v*z21 + 18*u*v^2*z21 + 3*v^3*z21 - 18*u^2*v*z22 - 18*u*v^2*z22 + 9*u^2*v*z23 - 9*u*v^2*z31 - 3*v^3*z31 + 9*u*v^2*z32 + v^3*z41 + 3*u^2*z11 + 9*u*v*z11 + 3*v^2*z11 - 6*u^2*z12 - 9*u*v*z12 + 3*u^2*z13 - 9*u*v*z21 - 6*v^2*z21 + 9*u*v*z22 + 3*v^2*z31 - 3*u*z11 - 3*v*z11 + 3*u*z12 + 3*v*z21 + z11

<h3 style="text-align: center;"> Example 1: Surface over regular X and Y</h3>
<p>This first example plays by very constrained rules for the X and Y values.  Essentially, but setting up regularly space samples between 0 and 3 on the X and Y axes the surface becomes a height - Z - over X and Y.  Note for this example the 16 control points are explicitly displayed in a 4 by 4 matrix in order to allow the relative position of the control points to be seen.  Then the actual 3d surface is plottted. </p>

In [4]:
GX = Matrix(SR, 4,4, ((0,1,2,3),(0,1,2,3),(0,1,2,3),(0,1,2,3)))
GY = Matrix(SR, 4,4, ((0,0,0,0),(1,1,1,1),(2,2,2,2),(3,3,3,3)))
GZ = Matrix(SR, 4,4, ((5,6,3,5),(3,5,6,3),(4,3,5,6),(5,4,3,5)))
QX = SV.transpose() * MB.transpose() * GX * MB * TV
QY = SV.transpose() * MB.transpose() * GY * MB * TV
QZ = SV.transpose() * MB.transpose() * GZ * MB * TV
PP = map(lambda x,y,z : Matrix(ZZ, 3,1, (x,y,z)), GX.list(), GY.list(), GZ.list())
MP = Matrix(SR, 4,4, PP); MP

[[0]
[0]
[5] [1]
[0]
[6] [2]
[0]
[3] [3]
[0]
[5]]
[[0]
[1]
[3] [1]
[1]
[5] [2]
[1]
[6] [3]
[1]
[3]]
[[0]
[2]
[4] [1]
[2]
[3] [2]
[2]
[5] [3]
[2]
[6]]
[[0]
[3]
[5] [1]
[3]
[4] [2]
[3]
[3] [3]
[3]
[5]]

In [5]:
gf = (QX[0][0], QY[0][0], QZ[0][0])
def cf(u,v): return (u + v) * 0.5
fig1  = sum(map(lambda x,y,z,c : point((x,y,z),color=c,size=16), GX.list(), GY.list(), GZ.list(),rainbow(16)))
fig1 += parametric_plot3d(gf,(u, 0.0,1.0),(v, 0.0, 1.0),color=(cf,colormaps.PiYG))
fig1.show()

Graphics3d Object

<h3 style="text-align: center;">Example 2: Height Field 2</h3>
<p>This is simpilar to Example 1 but with a bit more variation in height.</p>

In [6]:
GX = Matrix(SR, 4,4, ((0,1,2,3),(0,1,2,3),(0,1,2,3),(0,1,2,3)))
GY = Matrix(SR, 4,4, ((0,0,0,0),(1,1,1,1),(2,2,2,2),(3,3,3,3)))
GZ = Matrix(SR, 4,4, ((50,60,30,50),(30,50,60,30),(40,30,50,60),(50,40,30,50)))
QX = TV.transpose() * MB.transpose() * GX * MB * SV
QY = TV.transpose() * MB.transpose() * GY * MB * SV
QZ = TV.transpose() * MB.transpose() * GZ * MB * SV
PP = map(lambda x,y,z : Matrix(ZZ, 3,1, (x,y,z)), GX.list(), GY.list(), GZ.list())
MP = Matrix(SR, 4,4, PP); MP

[[ 0]
[ 0]
[50] [ 1]
[ 0]
[60] [ 2]
[ 0]
[30] [ 3]
[ 0]
[50]]
[[ 0]
[ 1]
[30] [ 1]
[ 1]
[50] [ 2]
[ 1]
[60] [ 3]
[ 1]
[30]]
[[ 0]
[ 2]
[40] [ 1]
[ 2]
[30] [ 2]
[ 2]
[50] [ 3]
[ 2]
[60]]
[[ 0]
[ 3]
[50] [ 1]
[ 3]
[40] [ 2]
[ 3]
[30] [ 3]
[ 3]
[50]]

In [7]:
gf = (QX[0][0], QY[0][0], QZ[0][0])
def cf(u,v): return (u + v) * 0.5
fig1  = sum(map(lambda x,y,z,c : point((x,y,z),color=c,size=16), GX.list(), GY.list(), GZ.list(),rainbow(16)))
fig1 += parametric_plot3d(gf,(u, 0.0,1.0),(v, 0.0, 1.0),color=(cf,colormaps.PiYG))
fig1.show()

Graphics3d Object

<h3 style="text-align: center;">Example 3: Pillow</h3>
<p>Here we begin to see the flexibility that comes with a Bezier Patch.  Just for example, realize that what you see in the next example cannot be expressed as a function of x an y.  In other words, for certain pairs (x,y) there are multiple values of z.</p>

In [8]:
GX = Matrix(SR, 4,4, ((2,1,4,3),(2,1,4,3),(2,1,4,3),(2,1,4,3)))
GY = Matrix(SR, 4,4, ((2,1,1,2),(2,1,1,2),(3,4,4,3),(3,4,4,3)))
GZ = Matrix(SR, 4,4, ((0,1,1,0),(1,2,2,1),(1,2,2,1),(0,1,1,0)))
QX = SV.transpose() * MB.transpose() * GX * MB * TV
QY = SV.transpose() * MB.transpose() * GY * MB * TV
QZ = SV.transpose() * MB.transpose() * GZ * MB * TV
PP = map(lambda x,y,z : Matrix(SR, 3,1, (Rational(x),Rational(y),Rational(z))), GX.list(), GY.list(), GZ.list())
MP = Matrix(SR, 4,4, PP); MP

[[2]
[2]
[0] [1]
[1]
[1] [4]
[1]
[1] [3]
[2]
[0]]
[[2]
[2]
[1] [1]
[1]
[2] [4]
[1]
[2] [3]
[2]
[1]]
[[2]
[3]
[1] [1]
[4]
[2] [4]
[4]
[2] [3]
[3]
[1]]
[[2]
[3]
[0] [1]
[4]
[1] [4]
[4]
[1] [3]
[3]
[0]]

In [9]:
gf = (QX[0][0], QY[0][0], QZ[0][0])
def cf(u,v): return (u + v) * 0.5
fig2  = sum(map(lambda x,y,z,c : point((x,y,z),color=c,size=16), GX.list(), GY.list(), GZ.list(),rainbow(16)))
fig2 += parametric_plot3d(gf,(u, 0.0,1.0),(v, 0.0, 1.0),color=(cf,colormaps.PiYG),opacity=0.5)
fig2.show()

Graphics3d Object

<h3 style="text-align: center;">Example 4: Tie the ends together</h3>
<p>Extending the previous example, this example proves a simple point: it is possible to bring all the ends of the curve to a single point in space.</p>

In [14]:
GX = Matrix(SR, 4,4, ((3,1, 4, 3),(2, 1, 4, 3),(2, 1, 4, 3),(2, 1, 4, 2)))
GY = Matrix(SR, 4,4, ((3,1, 1, 3),(2, 1, 1, 2),(3, 4, 4, 3),(2, 4, 4, 2)))
GZ = Matrix(SR, 4,4, ((0,  1, 1,   0),(1, 2, 2, 1),(1, 2, 2, 1),(0,   1, 1,   0)))
QX = SV.transpose() * MB.transpose() * GX * MB * TV
QY = SV.transpose() * MB.transpose() * GY * MB * TV
QZ = SV.transpose() * MB.transpose() * GZ * MB * TV
PP = map(lambda x,y,z : Matrix(SR, 3,1, (Rational(x),Rational(y),Rational(z))), GX.list(), GY.list(), GZ.list())
MP = Matrix(SR, 4,4, PP); MP

[[3]
[3]
[0] [1]
[1]
[1] [4]
[1]
[1] [3]
[3]
[0]]
[[2]
[2]
[1] [1]
[1]
[2] [4]
[1]
[2] [3]
[2]
[1]]
[[2]
[3]
[1] [1]
[4]
[2] [4]
[4]
[2] [3]
[3]
[1]]
[[2]
[2]
[0] [1]
[4]
[1] [4]
[4]
[1] [2]
[2]
[0]]

In [15]:
gf = (QX[0][0], QY[0][0], QZ[0][0])
def cf(u,v): return (u + v) * 0.5
fig3  = sum(map(lambda x,y,z,c : point((x,y,z),color=c,size=16), GX.list(), GY.list(), GZ.list(),rainbow(16)))
fig3 += parametric_plot3d(gf,(u, 0.0,1.0),(v, 0.0, 1.0),color=(cf,colormaps.PiYG))
fig3.show()

Graphics3d Object